In [149]:
#!pip install PyPDF2
import PyPDF2
import re

file = open('premiados_primeira_cerimonia_2018.pdf', 'rb')

pdf = PyPDF2.PdfFileReader(file)
pages = pdf.getNumPages()
texto = ""

for i in range(pages):
    texto += pdf.getPage(i).extractText()

texto = texto.replace('\n', ' ')
texto = texto.replace('\t', ' ')
texto = " ".join(texto.split())
texto = texto.split('[EMPRESA/INSTITUIÇÃO]')
texto = texto[1:]

empresa = texto[0].split('[PRÊMIO]')
premio = empresa[1].split('Os contemplados')

# algumas emrpesas possuem mais de 1 projeto fazer a verificacao a partir daqui (ultilizando o resto do premio :])

projeto = premio[1].split('[ PROJETO:')[1].split(']')

separator = re.findall('- [A-Z][A-Z] ', projeto[1])
cidade = projeto[1].split(separator[0])
alunos = cidade[1].split(" [Estudante] ")
orientadores = alunos.pop()


empresaE = empresa[0].strip()
premioE = premio[0].strip()
projetoE = projeto[0].split(']')[0].strip()
cidadeE = cidade[0].split(' De ')[1] + separator[0]

print("Empresa: " + empresaE)
print("Premio: " + premioE)
print("Projeto: " + projetoE)
print("Cidade: " + cidadeE)


Empresa: American Psychological Association
Premio: Outstanding for Behavioral Sciences
Projeto: Gra videz e maternidade na adolescência: mobilidade social e sociabilidade local na periferia de São Paulo
Cidade: SÃ O PAULO - SP 
